In [1]:
import numpy as np
import trimesh as tri
import torch

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from os import listdir

from utils import lddmm_utils, mesh_processing

[KeOps] Warning : cuda was detected, but driver API could not be initialized. Switching to cpu only.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define distances

In [2]:
# torch type and device
use_cuda = torch.cuda.is_available()
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
#torchdeviceId = "cpu"
torchdtype = torch.float32

# PyKeOps counterpart
KeOpsdeviceId = torchdeviceId.index  # id of Gpu device (in case Gpu is  used)
KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'

def varifold_dist(mesh1, mesh2, sigma):
    
    V1, F1 = np.array(mesh1.vertices), np.array(mesh1.faces)
    V2, F2 = np.array(mesh2.vertices), np.array(mesh2.faces)
    
    q0 = torch.from_numpy(V1).clone().detach().to(dtype=torchdtype, device=torchdeviceId).requires_grad_(True)
    VT = torch.from_numpy(V2).clone().detach().to(dtype=torchdtype, device=torchdeviceId)
    FS = torch.from_numpy(F1).clone().detach().to(dtype=torch.long, device=torchdeviceId)
    FT = torch.from_numpy(F2).clone().detach().to(dtype=torch.long, device=torchdeviceId)
    sigma = torch.tensor([sigma], dtype=torchdtype, device=torchdeviceId)
    
    p0 = torch.zeros(q0.shape, dtype=torchdtype, device=torchdeviceId, requires_grad=True)
    dataloss = lddmm_utils.lossVarifoldSurf(FS, VT, FT, lddmm_utils.GaussLinKernel(sigma=sigma))
    Kv = lddmm_utils.GaussKernel(sigma=sigma)
    loss = lddmm_utils.LDDMMloss(Kv, dataloss)

    dist = loss(p0, q0)

    return(dist)

In [3]:
mesh = tri.creation.uv_sphere()
V, F = torch.Tensor(mesh.vertices).to(dtype=torch.float, device = 'cpu'), torch.Tensor(mesh.faces).to(dtype=torch.int, device='cpu')

In [4]:
lddmm_utils.normVarifoldSurf(V,F, lddmm_utils.GaussLinKernel(sigma=0.1))

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.


tensor(0.3903)

### Distances shifted param

In [42]:
import os
list_files_ref = [str(i) + ".ply" for i in range(len(os.listdir("ref_base")))]
list_files_shifted = [str(i) + ".ply" for i in range(len(os.listdir("reparam_base")))]
list_files_unbalanced = [str(i) + ".ply" for i in range(len(os.listdir("bad_reparam_base")))]

In [45]:
list_dist_ref_shifted = []
sigma = 50
for i in range(len(os.listdir("ref_base"))):
    mesh1, mesh2 = tri.load_mesh(os.path.join("ref_base",list_files_ref[i])), tri.load_mesh(os.path.join("reparam_base",list_files_shifted[i]))
    V1, F1 = torch.Tensor(mesh1.vertices).to(dtype=torch.float, device='cpu'), torch.Tensor(mesh1.faces).to(dtype=torch.int, device='cpu')
    V2, F2 = torch.Tensor(mesh2.vertices).to(dtype=torch.float, device='cpu'), torch.Tensor(mesh2.faces).to(dtype=torch.int, device='cpu')
    list_dist_ref_shifted.append(lddmm_utils.lossVarifoldSurf(F2, V1, F1, lddmm_utils.GaussLinKernel(sigma=sigma))(V2) / lddmm_utils.normVarifoldSurf(V1,F1, lddmm_utils.GaussLinKernel(sigma=sigma)))

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

...

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.


In [46]:
np.array(list_dist_ref_shifted).mean()

1.0473696e-05

In [37]:
list_dist_ref_unbalanced = []
sigma = 5
for i in range(len(os.listdir("ref_base"))):
    mesh1, mesh2 = tri.load_mesh(os.path.join("ref_base",list_files_ref[i])), tri.load_mesh(os.path.join("bad_reparam_base",list_files_unbalanced[i]))
    V1, F1 = torch.Tensor(mesh1.vertices).to(dtype=torch.float, device='cpu'), torch.Tensor(mesh1.faces).to(dtype=torch.int, device='cpu')
    V2, F2 = torch.Tensor(mesh2.vertices).to(dtype=torch.float, device='cpu'), torch.Tensor(mesh2.faces).to(dtype=torch.int, device='cpu')
    dist = lddmm_utils.lossVarifoldSurf(F2, V1, F1, lddmm_utils.GaussLinKernel(sigma=sigma))(V2) / lddmm_utils.normVarifoldSurf(V1,F1, lddmm_utils.GaussLinKernel(sigma=sigma))
    list_dist_ref_unbalanced.append(dist)

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

...

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] Warning : keyword argument dtype in Genred is deprecated ; argument is ignored.
[pyKeOps] 

In [38]:
np.array(list_dist_ref_unbalanced).mean()

0.004448822

In [6]:
from chamfer_distance import ChamferDistance as chamfer_dist

In [14]:
list_dist_ref_shifted_chamfer = []

for i in range(len(os.listdir("ref_base"))):
    mesh1, mesh2 = tri.load_mesh(os.path.join("ref_base",list_files_ref[i])), tri.load_mesh(os.path.join("reparam_base",list_files_shifted[i]))
    V1 = torch.Tensor(mesh1.vertices).to(dtype=torch.float, device='cpu')
    V2 = torch.Tensor(mesh2.vertices).to(dtype=torch.float, device='cpu')
    dist1, dist2, idx1, idx2 = chamfer_dist(V1,V2)
    
    list_dist_ref_shifted.append((torch.mean(dist1)) + (torch.mean(dist2)))

TypeError: __init__() takes 1 positional argument but 3 were given